# **Proyecto Final Herramientas de BigData**

**Maestria en Analitica Aplicada**

**Universidad de La Sabana, Facultad de Ingeniería**


Presentado por: **Julián Andrés Santos Méndez**

email: ingjuliansantos@gmail.com

Archivo: Adquisición de datos con la API de XM S.A. E.S.P.

In [8]:
import pandas as pd
import datetime as dt
import calendar
import requests
import json

## **Extracción de datos de la REST API de XM S.A. E.S.P.**
---

En 2020, XM S.A. E.S.P. puso a disposición del público su API REST para facilitar la consulta de datos del SIN.

Para simplificar aún más el uso de la API, desarrollaron un módulo llamado *pydataxm.py*, que permite realizar consultas a la API de manera más directa y sencilla, devolviendo un *DataFrame* de Pandas con los datos de cada variable consultada.

In [9]:
import pydataxm as api_xm
consult = api_xm.ReadDB()

## Funciones de Apoyo

La API REST de XM tiene un límite máximo de 31 días (registros) por consulta. Para evitar errores al superar este límite, se han creado las siguientes funciones, las cuales permiten descargar un mes a la vez.

In [3]:
def generate_date_dict(start_year, end_year, end_month):
    date_dict = {}
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if year == end_year and month > end_month:
                break
            start_date = dt.date(year, month, 1)
            end_date = dt.date(year, month, calendar.monthrange(year, month)[1])
            date_dict[(year, month)] = {"start_date": start_date, "end_date": end_date}
    return date_dict

def extract_metric(date_dict, metric, num_metric):
    df_to_concat = pd.DataFrame()
    for key in date_dict:
        start_date = date_dict[key]['start_date']
        end_date = date_dict[key]['end_date']
        print(f"Start for: {key}: start_date = {date_dict[key]['start_date']}, end_date = {date_dict[key]['end_date']}")
        df = consult.request_data(metric, num_metric, start_date, end_date)
        print(f"Done for: {key}: start_date = {date_dict[key]['start_date']}, end_date = {date_dict[key]['end_date']}")
        df_to_concat = pd.concat([df_to_concat, df], ignore_index=True)
    return df_to_concat

## Extracción de Listados

### Listado de Metricas

In [18]:
url = "http://servapibi.xm.com.co/lists"
request = {
    "MetricId": "ListadoMetricas",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_metricas = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_metricas

,Id,Values.MetricId,Values.MetricName,Values.Entity,Values.MaxDays,Values.Type,Values.Url,Values.Filter,Values.MetricUnits,Values.MetricDescription
0,Sistema,DemaReal,Demanda Real por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,kWh,Demanda de usuarios regulados y no regulados q...
1,Sistema,DemaReal,Demanda Real por Agente,Agente,31,HourlyEntities,http://servapibi.xm.com.co/hourly,Codigo Comercializador,kWh,Demanda de usuarios regulados y no regulados q...
2,Sistema,ExpoMoneda,Exportaciones Moneda por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,COP,Transferencias de Energia desde Colombia hacia...
3,Sistema,DemaCome,Demanda Comercial por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,kWh,Considera la demanda propia de cada comerciali...
4,Sistema,Gene,Generación por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,kWh,Generacion neta de cada una de las plantas Nac...
...,...,...,...,...,...,...,...,...,...,...
182,Sistema,ListadoAgentes,Listado de agentes con atributos por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de Agentes registrados ante el ASIC co...
183,Sistema,ListadoRios,Listado de rios por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de ríos que se encuentran registrados ...
184,Sistema,ListadoEmbalses,Listado de embalses por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de embalses que se encuentran registra...
185,Sistema,ListadoMetricas,Listado de Métricas por Sistema,Sistema,731,ListsEntities,http://servapibi.xm.com.co/list,No aplica,,Listado de métricas disponibles en la API XM c...


In [19]:
listado_metricas.to_json('./data/listado_metricas.json', orient='records', lines=True)

### Listado de Plantas de Generación

In [20]:
request = {
    "MetricId": "ListadoRecursos",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_plantas = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_plantas

,Id,Values.Code,Values.Name,Values.Type,Values.Disp,Values.RecType,Values.CompanyCode,Values.EnerSource,Values.OperStartdate,Values.State
0,Sistema,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION
1,Sistema,2QEK,SALTO II,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,ENDG,AGUA,2014-06-25,OPERACION
2,Sistema,2QRL,LA REBUSCA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,GEN. DISTRIBUIDA,HZEG,AGUA,2014-07-24,OPERACION
3,Sistema,2QV2,BAJO TULUA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,EPSG,AGUA,2015-01-30,OPERACION
4,Sistema,2R22,LAGUNETA,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,ENDG,AGUA,2014-12-17,OPERACION
...,...,...,...,...,...,...,...,...,...,...
680,Sistema,VNTB,VENTANA B,HIDRAULICA,NO DESPACHADO CENTRALMENTE,NORMAL,HDTG,AGUA,1957-11-01,OPERACION
681,Sistema,ZPA2,ZIPAEMG 2,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1964-01-01,OPERACION
682,Sistema,ZPA3,ZIPAEMG 3,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1976-01-01,OPERACION
683,Sistema,ZPA4,ZIPAEMG 4,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1981-04-01,OPERACION


In [21]:
listado_plantas.to_json('./data/listado_recursos_plantas.json', orient='records', lines=True)

### Listado de Ríos

In [22]:
request = {
    "MetricId": "ListadoRios",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_rios = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_rios

,Id,Values.Code,Values.Name,Values.HydroRegion,Values.Status
0,Sistema,ALICBOGO,BOGOTA N.R.,CENTRO,ACTIVO
1,Sistema,ALTOANCH,ALTOANCHICAYA,VALLE,ACTIVO
2,Sistema,AMOYAMOY,AMOYA,CENTRO,ACTIVO
3,Sistema,BAJOANCH,DIGUA,VALLE,ACTIVO
4,Sistema,BETAMAG1,BETANIA CP,CENTRO,ACTIVO
5,Sistema,BOCATENC,CONCEPCION,ANTIOQUIA,ACTIVO
6,Sistema,CAL1CALM,CALIMA,VALLE,ACTIVO
7,Sistema,CAMECAMP,CAMPOALEGRE,CALDAS,ACTIVO
8,Sistema,CAMECHIN,CHINCHINA,CALDAS,ACTIVO
9,Sistema,CLLRPORC,CARLOS LLERAS,ANTIOQUIA,ACTIVO


In [23]:
listado_rios.to_json('./data/listado_rios.json', orient='records', lines=True)

### Listado Embalses

In [24]:
request = {
    "MetricId": "ListadoEmbalses",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_embalses = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_embalses

,Id,Values.Code,Values.Name,Values.HydroRegion,Values.Status
0,Sistema,AGREGADO,AGREGADO BOGOTA,CENTRO,ACTIVO
1,Sistema,ALTOANCH,ALTOANCHICAYA,VALLE,ACTIVO
2,Sistema,BETANIA,BETANIA,CENTRO,ACTIVO
3,Sistema,CALIMA1,CALIMA1,VALLE,ACTIVO
4,Sistema,CHUZA,CHUZA,ORIENTE,ACTIVO
5,Sistema,ELQUIMBO,EL QUIMBO,CENTRO,ACTIVO
6,Sistema,ESMERALD,ESMERALDA,ORIENTE,ACTIVO
7,Sistema,FLORIDA2,FLORIDA II,RIOS ESTIMADOS,ACTIVO
8,Sistema,GUAVIO,GUAVIO,ORIENTE,ACTIVO
9,Sistema,GUAVIO,GUAVIO,ORIENTE,ACTIVO


In [25]:
listado_embalses.to_json('./data/listado_embalses.json', orient='records', lines=True)

### Listado Empresas del Sistema Electrico

In [27]:
request = {
    "MetricId": "ListadoAgentes",
    "Entity": "Sistema"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_agentes = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_agentes

,Id,Values.Code,Values.Name,Values.Activity,Values.OperStartdate,Values.State
0,Sistema,GREC,GRUPO ENERGÉTICO SA ESP,COMERCIALIZACIÓN,2024-05-15,OPERACION
1,Sistema,SDEG,SOLAR EL EDÉN S.A.S. E.S.P.,GENERACIÓN,2024-05-07,OPERACION
2,Sistema,CPSG,EL CAMPANO SOLAR S.A.S. E.S.P.,GENERACIÓN,2024-04-30,OPERACION
3,Sistema,SHAG,OPERADORA SHANGRI-LA S.A.S E.S.P,GENERACIÓN,2024-04-30,OPERACION
4,Sistema,CSMG,SOLAR VILLAVIEJA S.A.S. E.S.P.,GENERACIÓN,2024-04-29,OPERACION
...,...,...,...,...,...,...
612,Sistema,CTGC,EMPRESAS MUNICIPALES DE CARTAGO S.A. E.S.P.,COMERCIALIZACIÓN,1995-07-20,INACTIVO
613,Sistema,CTGD,EMPRESAS MUNICIPALES DE CARTAGO S.A. E.S.P.,DISTRIBUCIÓN,1995-07-20,INACTIVO
614,Sistema,EPPC,EMPRESAS PUBLICAS DE PEREIRA S.A. E.S.P.,COMERCIALIZACIÓN,1995-07-20,INACTIVO
615,Sistema,EPMT,EMPRESAS PUBLICAS DE MEDELLIN E.S.P.,TRANSPORTE,1995-01-01,OPERACION


In [29]:
listado_agentes.to_json('./data/listado_empresas_agentes.json', orient='records', lines=True)

### Listado de Autogeneradores

In [32]:
request = {
    "MetricId": "ListadoAGPE",
    "Entity": "Agente"
}
connection = requests.post(url, json=request)
data_json = json.loads(connection.content)
listado_autogeneradores = pd.json_normalize(data_json['Items'], 'ListEntities')
listado_autogeneradores

,Id,Values.AgpeID,Values.Name,Values.CompanyCode,Values.ComZone,Values.VoltageLevel,Values.EnerSource,Values.InstalledCapacity
0,Agente,Agp00001,PROMOTORA RD LTDA,CMMC,CARIBE MAR,2,RAD SOLAR,9.000000
1,Agente,Agp00002,ORGANIZACION TERPEL S A,EPSC,VALLE DEL CAUCA,2,GAS,16.011109
2,Agente,Agp00003,ANDREA BARRIOS,CNSC,NORTE DE SANTANDER,1,RAD SOLAR,5.000000
3,Agente,Agp00004,JUAN JOSE SANTA VALLA,CMMC,CARIBE MAR,2,RAD SOLAR,10.000000
4,Agente,Agp00005,LUIS ALFONSO VILLAMIL VARGAS,CSSC,CARIBE SOL,1,RAD SOLAR,3.000000
...,...,...,...,...,...,...,...,...
8787,Agente,Grp0988,AGPE FUNDACION OFTALMOLOGICA,BIAC,NARIÑO,2,RAD SOLAR,20.000000
8788,Agente,Grp0993,AGPE RAMON GIRALDO GONZALEZ,SCEC,CARIBE MAR,1,RAD SOLAR,27.000000
8789,Agente,Unt0499,AGPE - ECOPETROL LA HORMIGA,GECG,BAJO PUTUMAYO,3,GAS,1000.000000
8790,Agente,Unt0514,AGPE ENTREPALMAS,EPSG,META,2,BIOGAS,500.000000


In [47]:
listado_autogeneradores.to_json('./data/listado_autogeneradores.json', orient='records', lines=True)

## Extracción de Variables de Generación

### Generación del Sistema por Hora

In [46]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "Gene"
num_metric = 0
df_generacion_sistema = extract_metric(date_dict,metric,num_metric)
df_generacion_sistema

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,5609982.23000,5296750.71000,4919467.58000,4638776.22000,4442074.76000,4279026.64000,3827254.47000,3832573.95000,...,4391372.59000,4365640.95000,4605721.12000,6021430.06000,6527391.35000,6261393.45000,5776398.57000,5211439.50000,4714078.01000,2010-01-01
1,Sistema,Sistema,4443442.83000,4238022.31000,4153685.87000,4103574.79000,4153986.60000,4252501.43000,4185993.85000,4479587.60000,...,5397357.88000,5317214.52000,5482854.99000,6895377.37000,7356071.75000,7109606.16000,6611477.08000,5931707.62000,5211930.81000,2010-01-02
2,Sistema,Sistema,4771417.74000,4520041.29000,4364500.03000,4210941.04000,4205070.55000,4312551.86000,4156670.92000,4331637.24000,...,5015866.10000,4981069.04000,5169045.72000,6637669.36000,7126930.30000,6924827.71000,6432570.74000,5752050.08000,5135978.48000,2010-01-03
3,Sistema,Sistema,4705250.02000,4493314.06000,4399068.67000,4395569.56000,4475996.61000,4777750.57000,4911096.21000,5322214.49000,...,6618343.03000,6458778.34000,6529860.50000,7819390.86000,8232402.15000,7884258.05000,7197982.57000,6288573.58000,5535400.79000,2010-01-04
4,Sistema,Sistema,5171964.92000,4939629.25000,4840107.60000,4774763.87000,4949684.01000,5195571.83000,5362721.47000,5742326.06000,...,6778398.35000,6715045.20000,6745497.24000,8023899.14000,8336888.23000,7929022.67000,7243496.48000,6378988.75000,5737144.04000,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,Sistema,Sistema,8518210.80000,8198137.14000,7966493.50000,7848538.15000,7993954.30000,8373795.23000,8533971.50000,8989609.97000,...,10224371.17000,10138968.61000,9944771.52000,10267157.48000,10388113.98000,10186516.32000,9774795.94000,9269233.15000,8783850.22000,2024-04-26
5230,Sistema,Sistema,8416268.81000,8139684.03000,7934365.14000,7801216.87000,7802711.66000,7866953.71000,8060693.66000,8611512.50000,...,9613448.98000,9515583.90000,9453804.19000,10022279.38000,10070282.41000,9806988.02000,9391935.06000,8934686.71000,8515651.18000,2024-04-27
5231,Sistema,Sistema,8153358.39000,7823646.09000,7495560.68000,7290789.06000,7186795.65000,7058942.05000,6899473.59000,7167562.93000,...,8373980.81000,8391670.64000,8366776.05000,9037799.05000,9296771.60000,9194946.81000,8963686.66000,8545982.69000,8101750.18000,2024-04-28
5232,Sistema,Sistema,7755006.39000,7559477.81000,7424995.07000,7358043.41000,7541190.63000,7993483.44000,8218854.74000,8737461.99000,...,10365625.70000,10314129.94000,10040895.25000,10521997.09000,10740452.13000,10521840.29000,10125689.57000,9473347.41000,8842359.25000,2024-04-29


In [48]:
df_generacion_sistema.to_json('./data/generacion_sistema.json', orient='records', lines=True)

### Generacion por Planta Recurso

In [49]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "Gene"
num_metric = 1
df_generacion_planta_recurso = extract_metric(date_dict,metric,num_metric)
df_generacion_planta_recurso

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Recurso,AFR1,4116,4027,4069,4109,3990,4158,3887,3781,...,3814,3673,3754,3803,3828,3581,3679,3803,3613,2010-01-01
1,Recurso,ALBG,14824,14817,14804,14804,14813,13686,13098,17048,...,11080,11071,11055,11069,11086,11096,11098,11091,11049,2010-01-01
2,Recurso,AMF1,622,622,622,622,622,620,620,621,...,634,634,633,632,633,632,632,633,633,2010-01-01
3,Recurso,AMR1,2,3,5,3,,,,31,...,,,,,,,,,,2010-01-01
4,Recurso,ASN1,486,486,490,484,488,485,488,487,...,487,486,486,487,487,487,488,487,487,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817316,Recurso,VNTB,986.33,888.42,1065.41,704.19,1291.19,1545.66,1599.5,1662.4,...,2406.07,2389.54,2385.53,2357.29,2291.29,2340.3,2349.94,2317.12,2028.55,2024-04-30
817317,Recurso,ZPA2,29919.99,29953.03,29911.08,29938.3,29881.28,29941.76,29883,29952.68,...,29559.6,29554.02,29579.26,29572.7,29464.75,28800.29,29303.37,29623.16,29640.7,2024-04-30
817318,Recurso,ZPA3,,,,,,,,,...,,,,,,2829.99,17682.36,25107.14,30922.03,2024-04-30
817319,Recurso,ZPA4,60652.2,60717.28,60783.27,60703.95,61031.19,61028.64,61098.96,61142.5,...,61592.99,61363.92,61480.96,61260.58,61578.56,61480.5,61023.61,61048.03,61602.61,2024-04-30


In [50]:
df_generacion_planta_recurso.to_json('./data/generacion_planta_recurso.json', orient='records', lines=True)

### Generación Ideal Sistema

In [56]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "GeneIdea"
num_metric = 0
df_generacion_ideal_sistema = extract_metric(date_dict,metric,num_metric)
df_generacion_ideal_sistema

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,5609982.23000,5299387.09000,4922337.76000,4640952.55000,4443431.20000,4280072.11000,3828774.82000,3832979.13000,...,4392140.29000,4366427.63000,4606016.29000,6021430.06000,6527391.35000,6263172.04000,5779281.34000,5212138.25000,4714247.65000,2010-01-01
1,Sistema,Sistema,4445348.51000,4238116.21000,4154887.32000,4103709.79000,4153986.60000,4252501.43000,4186498.45000,4480615.46000,...,5397357.88000,5318190.02000,5484348.49000,6895377.37000,7356107.90000,7110891.37000,6612818.22000,5934135.44000,5214100.60000,2010-01-02
2,Sistema,Sistema,4772268.38000,4520104.69000,4365723.51000,4211679.57000,4205364.41000,4312551.86000,4156670.92000,4333251.97000,...,5016125.95000,4981354.24000,5169667.20000,6637669.36000,7126930.30000,6927793.22000,6433970.64000,5753971.20000,5137059.49000,2010-01-03
3,Sistema,Sistema,4705845.03000,4493923.68000,4400230.86000,4396643.54000,4475996.61000,4778560.19000,4912357.20000,5323877.68000,...,6517107.95000,6357582.34000,6428430.50000,7712972.01000,8130900.15000,7782972.05000,7096426.57000,6186918.31000,5434024.79000,2010-01-04
4,Sistema,Sistema,5044349.31000,4810224.85000,4711179.46000,4644547.47000,4821049.57000,5066868.38000,5233620.74000,5612777.12000,...,6648597.90000,6586555.45000,6617438.28000,7888852.40000,8206986.44000,7800736.46000,7115549.68000,6250156.98000,5605618.05000,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,Sistema,Sistema,8518210.80000,8198137.14000,7966493.50000,7848538.15000,7993954.30000,8373795.23000,8533971.50000,8989609.97000,...,10224371.17000,10138968.61000,9944771.52000,10267157.48000,10388113.98000,10186516.32000,9774795.94000,9269233.15000,8783850.22000,2024-04-26
5230,Sistema,Sistema,8416268.81000,8139684.03000,7934365.14000,7801216.87000,7802711.66000,7866953.71000,8060693.66000,8611512.50000,...,9613448.98000,9515583.90000,9453804.19000,10022279.38000,10070282.41000,9806988.02000,9391935.06000,8934686.71000,8515651.18000,2024-04-27
5231,Sistema,Sistema,8153358.39000,7823646.09000,7495560.68000,7290789.06000,7186795.65000,7058942.05000,6899473.59000,7167562.93000,...,8373980.81000,8391670.64000,8366776.05000,9037799.05000,9296771.60000,9195967.42000,8965875.21000,8547088.54000,8101750.18000,2024-04-28
5232,Sistema,Sistema,7755006.39000,7559477.81000,7424995.07000,7358043.41000,7541190.63000,7993483.44000,8218854.74000,8737461.99000,...,10365625.70000,10315355.57000,10040895.25000,10521997.09000,10741050.68000,10522650.66000,10127382.98000,9476883.76000,8842359.25000,2024-04-29


In [57]:
df_generacion_ideal_sistema.to_json('./data/generacion_ideal_sistema.json', orient='records', lines=True)

### Generacion Ideal Planta Recurso

In [60]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "GeneIdea"
num_metric = 1
df_generacion_ideal_planta_recurso = extract_metric(date_dict,metric,num_metric)
df_generacion_ideal_planta_recurso

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Recurso,AFR1,4116,4027,4069,4109,3990,4158,3887,3781,...,3814,3673,3754,3803,3828,3581,3679,3803,3613,2010-01-01
1,Recurso,ALBG,11000,11000,11000,11000,11000,11000,11000,11000,...,11000,11000,11000,11000,11000,11000,11000,11000,11000,2010-01-01
2,Recurso,AMF1,622,622,622,622,622,620,620,621,...,634,634,633,632,633,632,632,633,633,2010-01-01
3,Recurso,AMR1,2,3,5,3,,,,31,...,,,,,,,,,,2010-01-01
4,Recurso,ASN1,486,486,490,484,488,485,488,487,...,487,486,486,487,487,487,488,487,487,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804266,Recurso,VLT1,10038.6,10010,9873.6,9015.6,8199.4,9752.6,10060.6,10025.4,...,10065,10828.4,10608.4,10538,10164,8329.2,4833.4,5537.4,4496.8,2024-04-30
804267,Recurso,VNTB,986.33,888.42,1065.41,704.19,1291.19,1545.66,1599.5,1662.4,...,2406.07,2389.54,2385.53,2357.29,2291.29,2340.3,2349.94,2317.12,2028.55,2024-04-30
804268,Recurso,ZPA2,28000,26000,24000,9000,,,,,...,,,,,,,,,,2024-04-30
804269,Recurso,ZPA4,59000,57000,55000,53000,51000,49000,47000,16000,...,,,,,,,,,,2024-04-30


In [61]:
df_generacion_ideal_planta_recurso.to_json('./data/generacion_ideal_planta_recurso.json', orient='records', lines=True)

## Extracción de Variables Capacidad y Volumen

### Volumen Util del Sistema en Unidades de Energía

In [64]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "VoluUtilDiarEner"
num_metric = 0
df_volumen_util_energia_sistema = extract_metric(date_dict,metric,num_metric)
df_volumen_util_energia_sistema

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Value,Date
0,Sistema,10006614779.00000,2010-01-01
1,Sistema,9995146811.50000,2010-01-02
2,Sistema,9992677303.10000,2010-01-03
3,Sistema,9965341213.60000,2010-01-04
4,Sistema,9926759570.30000,2010-01-05
...,...,...,...
5229,Sistema,5399083900.00000,2024-04-26
5230,Sistema,5454064900.00000,2024-04-27
5231,Sistema,5557993200.00000,2024-04-28
5232,Sistema,5672509900.00000,2024-04-29


In [65]:
df_volumen_util_energia_sistema.to_json('./data/volumen_util_energia_sistema.json', orient='records', lines=True)

### Volumen Util por Planta Recurso en Unidades de Energía

In [66]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "VoluUtilDiarEner"
num_metric = 1
df_volumen_util_energia_planta = extract_metric(date_dict,metric,num_metric)
df_volumen_util_energia_planta

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Name,Value,Date
0,Embalse,AGREGADO BOGOTA,2.046298e+09,2010-01-01
1,Embalse,ALTOANCHICAYA,1.712760e+07,2010-01-01
2,Embalse,AMANI,2.007019e+08,2010-01-01
3,Embalse,BETANIA,1.252323e+08,2010-01-01
4,Embalse,CALIMA1,1.722852e+08,2010-01-01
...,...,...,...,...
116514,Embalse,SALVAJINA,2.291914e+08,2024-04-30
116515,Embalse,SAN LORENZO,3.830000e+07,2024-04-30
116516,Embalse,TOPOCORO,3.250895e+08,2024-04-30
116517,Embalse,TRONERAS,2.472430e+07,2024-04-30


In [67]:
df_volumen_util_energia_planta.to_json('./data/volumen_util_energia_planta.json', orient='records', lines=True)

### Capacidad Util Sistema Unidades Energia

In [70]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "CapaUtilDiarEner"
num_metric = 0
df_capacidad_util_energia = extract_metric(date_dict,metric,num_metric)
df_capacidad_util_energia

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Value,Date
0,Sistema,15428396811.00000,2010-01-01
1,Sistema,15428396811.00000,2010-01-02
2,Sistema,15428396811.00000,2010-01-03
3,Sistema,15428396811.00000,2010-01-04
4,Sistema,15428396811.00000,2010-01-05
...,...,...,...
5229,Sistema,17359561885.00000,2024-04-26
5230,Sistema,17359561885.00000,2024-04-27
5231,Sistema,17359561885.00000,2024-04-28
5232,Sistema,17359561885.00000,2024-04-29


In [71]:
df_capacidad_util_energia.to_json('./data/capacidad_util_energia.json', orient='records', lines=True)

### Capacidad Util Energia por Embalse

In [74]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "CapaUtilDiarEner"
num_metric = 1
df_capacidad_util_energia_embalse = extract_metric(date_dict,metric,num_metric)
df_capacidad_util_energia_embalse

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Name,Value,Date
0,Embalse,AGREGADO BOGOTA,4.447220e+09,2010-01-01
1,Embalse,ALTOANCHICAYA,3.721801e+07,2010-01-01
2,Embalse,AMANI,2.303661e+08,2010-01-01
3,Embalse,BETANIA,1.426662e+08,2010-01-01
4,Embalse,CALIMA1,2.035201e+08,2010-01-01
...,...,...,...,...
116656,Embalse,SALVAJINA,5.295982e+08,2024-04-30
116657,Embalse,SAN LORENZO,4.251428e+08,2024-04-30
116658,Embalse,TOPOCORO,1.033507e+09,2024-04-30
116659,Embalse,TRONERAS,6.384768e+07,2024-04-30


In [75]:
df_capacidad_util_energia_embalse.to_json('./data/capacidad_util_energia_embalse.json', orient='records', lines=True)

### Capacidad Efectiva Neta por Recurso

In [82]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "CapEfecNeta"
num_metric = 1
df_capacidad_efectiva_neta = extract_metric(date_dict,metric,num_metric)
df_capacidad_efectiva_neta

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Code,Value,Date
0,Recurso,2QFU,4000.00000,2010-01-01
1,Recurso,ABJ1,450.00000,2010-01-01
2,Recurso,AFR1,7050.00000,2010-01-01
3,Recurso,ALBG,429000.00000,2010-01-01
4,Recurso,AMF1,810.00000,2010-01-01
...,...,...,...,...
1153572,Recurso,VNTB,2500.00000,2024-04-30
1153573,Recurso,ZPA2,36000.00000,2024-04-30
1153574,Recurso,ZPA3,63000.00000,2024-04-30
1153575,Recurso,ZPA4,64000.00000,2024-04-30


In [83]:
df_capacidad_efectiva_neta.to_json('./data/capacidad_efectiva_neta.json', orient='records', lines=True)

### Aportes de Energia del Sistema

In [86]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "AporEner"
num_metric = 0
df_aportes_energia_sistema = extract_metric(date_dict,metric,num_metric)
df_aportes_energia_sistema

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Value,Date
0,Sistema,45616100.00000,2010-01-01
1,Sistema,43097100.00000,2010-01-02
2,Sistema,43400400.00000,2010-01-03
3,Sistema,40410400.00000,2010-01-04
4,Sistema,41190600.00000,2010-01-05
...,...,...,...
5229,Sistema,175197100.00000,2024-04-26
5230,Sistema,190925700.00000,2024-04-27
5231,Sistema,206187700.00000,2024-04-28
5232,Sistema,303206200.00000,2024-04-29


In [87]:
df_aportes_energia_sistema.to_json('./data/aportes_energia_sistema.json', orient='records', lines=True)

### Aportes de Energia del Sistema por cada Río

In [90]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "AporEner"
num_metric = 1
df_aportes_energia_sistema_rio = extract_metric(date_dict,metric,num_metric)
df_aportes_energia_sistema_rio

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Name,Value,Date
0,Rio,A. SAN LORENZO,4352200.0,2010-01-01
1,Rio,ALTOANCHICAYA,3209600.0,2010-01-01
2,Rio,BATA,1461000.0,2010-01-01
3,Rio,CAUCA SALVAJINA,2867400.0,2010-01-01
4,Rio,CHUZA,296100.0,2010-01-01
...,...,...,...,...
158563,Rio,SAN FRANCISCO,360300.0,2024-04-30
158564,Rio,SAN MIGUEL,1111500.0,2024-04-30
158565,Rio,SINU URRA,4322300.0,2024-04-30
158566,Rio,SOGAMOSO,17014300.0,2024-04-30


In [91]:
df_aportes_energia_sistema_rio.to_json('./data/aportes_energia_sistema_rio.json', orient='records', lines=True)

### Aportes Media Historica Sistema

In [98]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "AporEnerMediHist"
num_metric = 0
df_aportes_energia_sistema_media_historica = extract_metric(date_dict,metric,num_metric)
df_aportes_energia_sistema_media_historica

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Value,Date
0,Sistema,75130000.00000,2010-01-01
1,Sistema,75130000.00000,2010-01-02
2,Sistema,75130000.00000,2010-01-03
3,Sistema,75130000.00000,2010-01-04
4,Sistema,75130000.00000,2010-01-05
...,...,...,...
5229,Sistema,222790000.00000,2024-04-26
5230,Sistema,222790000.00000,2024-04-27
5231,Sistema,222790000.00000,2024-04-28
5232,Sistema,222790000.00000,2024-04-29


In [99]:
df_aportes_energia_sistema_media_historica.to_json('./data/aportes_energia_sistema_media_historica.json', orient='records', lines=True)

### Aportes Media Historica Río

In [100]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = "AporEnerMediHist"
num_metric = 1
df_aportes_energia_rio_media_historica = extract_metric(date_dict,metric,num_metric)
df_aportes_energia_rio_media_historica

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Name,Value,Date
0,Rio,A. SAN LORENZO,5520000.0,2010-01-01
1,Rio,ALTOANCHICAYA,4370000.0,2010-01-01
2,Rio,BATA,2640000.0,2010-01-01
3,Rio,BLANCO,210000.0,2010-01-01
4,Rio,BOGOTA N.R.,3720000.0,2010-01-01
...,...,...,...,...
167232,Rio,SAN FRANCISCO,340000.0,2024-04-30
167233,Rio,SAN MIGUEL,1780000.0,2024-04-30
167234,Rio,SINU URRA,2820000.0,2024-04-30
167235,Rio,SOGAMOSO,19000000.0,2024-04-30


In [101]:
df_aportes_energia_rio_media_historica.to_json('./data/aportes_energia_rio_media_historica.json', orient='records', lines=True)

### Aportes Caudales Ríos

In [10]:
date_dict = generate_date_dict(2010, 2024, 4)
metric = 'AporCaudal'
num_metric = 0
df_aportes_caudal = extract_metric(date_dict,metric,num_metric)
df_aportes_caudal

Start for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Done for: (2010, 1): start_date = 2010-01-01, end_date = 2010-01-31
Start for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Done for: (2010, 2): start_date = 2010-02-01, end_date = 2010-02-28
Start for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Done for: (2010, 3): start_date = 2010-03-01, end_date = 2010-03-31
Start for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Done for: (2010, 4): start_date = 2010-04-01, end_date = 2010-04-30
Start for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Done for: (2010, 5): start_date = 2010-05-01, end_date = 2010-05-31
Start for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Done for: (2010, 6): start_date = 2010-06-01, end_date = 2010-06-30
Start for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Done for: (2010, 7): start_date = 2010-07-01, end_date = 2010-07-31
Start for: (2010, 8): start_date = 2010-0

,Id,Name,Value,Date
0,Rio,A. SAN LORENZO,18.810,2010-01-01
1,Rio,ALTOANCHICAYA,31.470,2010-01-01
2,Rio,BATA,8.730,2010-01-01
3,Rio,BLANCO,0.000,2010-01-01
4,Rio,BOGOTA N.R.,0.000,2010-01-01
...,...,...,...,...
163711,Rio,SAN FRANCISCO,4.444,2024-04-30
163712,Rio,SAN MIGUEL,32.920,2024-04-30
163713,Rio,SINU URRA,374.030,2024-04-30
163714,Rio,SOGAMOSO,513.420,2024-04-30


In [11]:
df_aportes_caudal.to_json('./data/aportes_caudal.json', orient='records', lines=True)